# Библиотеки

In [1]:
import pandas as pd
import tweepy as tw 
import praw
from praw.models import MoreComments
import snscrape.modules.twitter as sntwitter
import datetime
from tqdm.notebook import tqdm



from IPython.core.display import display, HTML
display(HTML('<style>.container {width: 100% !important; } </style>'))

# Парсинг

## Reddit

In [2]:
reddit = praw.Reddit(client_id='gYuNiBS7_keBx3UmPQh-Nw', client_secret='I3oLaghUnEk9IgmSRAhsGrp3QKanNQ', user_agent='ParsBars')

posts_total = []
subreddit_list = ['Bitcoin','Crypto','Cryptocurrency','Cryptocurrencies','Blockchain','Binance']

for x in subreddit_list:
    posts = []
    ml_subreddit = reddit.subreddit(x).hot(limit=1000000)
    sub_c = reddit.subreddit(x)
    sub_c = sub_c.subscribers
    
    for post in tqdm(ml_subreddit):
        posts_loc = []
        posts_loc.append([post.author,post.title, post.score, post.id, post.subreddit, post.num_comments, post.selftext, datetime.datetime.fromtimestamp(post.created)])
        submission = reddit.submission(id=post.id)
        comm = []
        scor = []
        aut = []
        tim = []
        submission.comments.replace_more(limit=0)
        for top_level_comment in submission.comments:
            comm += [top_level_comment.body]
            scor += [top_level_comment.score]
            aut += [top_level_comment.author]
            tim += [datetime.datetime.fromtimestamp(top_level_comment.created)]
        df = pd.DataFrame([comm,scor,aut,tim],index=['comment','comment_score','aut','tim']).T.sort_values(by='comment_score',ascending=False).iloc[:10]
        shap = min(df.shape[0],10)
        comm_df = pd.DataFrame(df.iloc[:,0].tolist()+df.iloc[:,1].tolist()+df.iloc[:,2].tolist() + df.iloc[:,3].tolist() \
                               + [sum(scor),sub_c],index= ['top {} comment'.format(x) for x in range(1,shap+1)]\
                               + ['top {} comment score'.format(x) for x in range(1,shap+1)] \
                               + ['top {} comment author'.format(x) for x in range(1,shap+1)] \
                               + ['top {} comment time'.format(x) for x in range(1,shap+1)] \
                               + ['total comments score','subreddit subscribers']).T
        posts +=[pd.concat([pd.DataFrame(posts_loc,columns=['author','title', 'score', 'id', 'subreddit', 'num_comments', 'body', 'created']),comm_df],axis=1)]
    
    posts = pd.concat(posts)
    posts_total += [posts]
posts_total = pd.concat(posts_total).to_csv('reddit.csv')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## Twitter

In [ ]:
maxTweets = 1000000

#snscrape.modules.twitter.TwitterHashtagScraper('Bitcoin')
#sntwitter.TwitterSearchScraper('Bitcoin').get_items()
posts_total = []
subreddit_list = ['Bitcoin','Crypto','Cryptocurrency','Cryptocurrencies','Blockchain','Binance']
for x in subreddit_list:
    tweets_list = []
    for i,tweet in tqdm(enumerate(sntwitter.TwitterSearchScraper('Bitcoin').get_items())):
        if i>maxTweets:
            break
        posts_total.append({'date':tweet.date, 
                             'id':tweet.id, 
                             'content':tweet.content, 
                             'username':tweet.username})
    
    tweets_df = pd.DataFrame(tweets_list)
    tweets_df['subreddit'] = x
    posts_total += [tweets_df]
posts_total = pd.concat(posts_total).to_csv('twitter.csv')

0it [00:00, ?it/s]